## Segmenting and Clustering Neighborhoods in Toronto

#### Scrape neighbourhoods table from Wikipedia with BeautifulSoup

In [42]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XsmZgApAMM4AApd8vuoAAACN","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":958430791,"wgRevisionId":958430791,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario

#### create dataframe

In [3]:
# find all rows
rows = soup.find('table').find_all('tr')

# rows list
row = [r.get_text() for r in rows]

In [4]:
#Converting content of PostalCode HTML table as dataframe

df = pd.DataFrame(row)
df = df[0].str.split('\n', expand=True)
df = df.rename(columns=df.iloc[0])
df = df.drop(df.index[0])
df.head()

,,Postal Code,,Borough,,Neighborhood,
1,,M1A,,Not assigned,,,
2,,M2A,,Not assigned,,,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,"Regent Park, Harbourfront",


#### Cleanse data

In [5]:
# Remove not assigned Borough
df1 = df[df.Borough != 'Not assigned']
df1.head()

,,Postal Code,,Borough,,Neighborhood,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,"Regent Park, Harbourfront",
6,,M6A,,North York,,"Lawrence Manor, Lawrence Heights",
7,,M7A,,Downtown Toronto,,"Queen's Park, Ontario Provincial Government",


#### Group Neighborhood by postal code

In [6]:
df1 = df1.groupby(['Postal Code', 'Borough'], sort = False).agg(','.join)
df1.reset_index(inplace = True)
df1

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [7]:
# Replace 'Not assigned' Neighborhood names with Borough name
df1.Neighborhood.replace("Not assigned",df.Borough,inplace=True)
df1

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [8]:
df1.shape

(103, 3)

#### Read Geospatial Data from csv file

In [9]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /Users/cheungoliver/anaconda3

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:           1.22.0-pyh9f0ad1d_0 conda-forge

The following packages will be UPDATED:

    ca-certificates: 2020.1.1-0          anaconda    --> 2020.

In [12]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df2 = pd.merge(df1, coordinates, on='Postal Code')
df2.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Create map and markers

work with only boroughs that contain the word Toronto

In [30]:
# Toronto=df2[df2['Borough'].str.contains('Toronto')]
# Toronto.head()

column_names = ["Postal Code", "Borough", "Neighborhood", "Latitude", "Longitude"]
Toronto = pd.DataFrame(columns=column_names)

Toronto_postal_codes = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in Toronto_postal_codes:
    Toronto = Toronto.append(df2[df2["Postal Code"]==postcode], sort=False, ignore_index=True)
    
Toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


In [31]:
import folium
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Borough'], Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)
map_toronto

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Explore and cluster the neighborhoods in Toronto

In [32]:
borough_names = list(df2.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto']

In [35]:
# create a new DataFrame with only boroughs that contain the word Toronto
df2 = df2[df2['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
df2.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [36]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

 **Foursquare** API

In [38]:
# Define Foursquare Credentials and Version
CLIENT_ID = '0UXRISZIO2EIL2VSIQ1TGAWUQC20EFDBVQV1PDYLO3CRLGG0' # Put Your Client Id
CLIENT_SECRET = 'P4LDZHRUHVTGCZHS5QEA0OYKZMNGXHDVATXQ01414REYYCZB' # Put You Client Secret 
VERSION = '20180604'
radius = 500
LIMIT = 100

In [43]:
# Foursquare API
venues = []

for lat, long, post, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Postal Code'], df2['Borough'], 
                                                  df2['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id=ZYLC4Q3I000O4R32DVJWJJTOTHCGC4O02TXYEPLDAS211SPQ&client_secret=OPKHF1MTRWKRHVR2DAV0IT1IK2H2XZDXJYTCNHVY5L44T55H&v=20180605 \
     &ll=43.653963,-79.387207&radius=500&limit=100".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [44]:
venues_df = pd.DataFrame(venues)


venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

venues_df.head()

(1794, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Textile Museum of Canada,43.654396,-79.386500,Art Museum
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Cafe Plenty,43.654571,-79.389450,Café
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Rolltation,43.654918,-79.387424,Japanese Restaurant
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Sansotei Ramen 三草亭,43.655157,-79.386501,Ramen Restaurant


Venue Counts

In [45]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,46,46,46,46,46,46
M4K,East Toronto,"The Danforth West, Riverdale",46,46,46,46,46,46
M4L,East Toronto,"India Bazaar, The Beaches West",46,46,46,46,46,46
M4M,East Toronto,Studio District,46,46,46,46,46,46
M4N,Central Toronto,Lawrence Park,46,46,46,46,46,46
M4P,Central Toronto,Davisville North,46,46,46,46,46,46
M4R,Central Toronto,"North Toronto West, Lawrence Park",46,46,46,46,46,46
M4S,Central Toronto,Davisville,46,46,46,46,46,46
M4T,Central Toronto,"Moore Park, Summerhill East",46,46,46,46,46,46


Analyze each area

In [46]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1794, 38)


,PostalCode,Borough,Neighborhoods,Art Gallery,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop,Café,...,Poke Place,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Seafood Restaurant,Smoke Shop,Sushi Restaurant,University,Vegetarian / Vegan Restaurant
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

2. Analyze Each Borough Neighborhood

In [47]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 38)


,PostalCode,Borough,Neighborhoods,Art Gallery,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop,Café,...,Poke Place,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Seafood Restaurant,Smoke Shop,Sushi Restaurant,University,Vegetarian / Vegan Restaurant
0,M4E,East Toronto,The Beaches,0.065217,0.021739,0.021739,0.021739,0.021739,0.021739,0.065217,...,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739,0.043478,0.021739,0.021739
1,M4K,East Toronto,"The Danforth West, Riverdale",0.065217,0.021739,0.021739,0.021739,0.021739,0.021739,0.065217,...,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739,0.043478,0.021739,0.021739
2,M4L,East Toronto,"India Bazaar, The Beaches West",0.065217,0.021739,0.021739,0.021739,0.021739,0.021739,0.065217,...,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739,0.043478,0.021739,0.021739
3,M4M,East Toronto,Studio District,0.065217,0.021739,0.021739,0.021739,0.021739,0.021739,0.065217,...,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739,0.043478,0.021739,0.021739
4,M4N,Central Toronto,Lawrence Park,0.065217,0.021739,0.021739,0.021739,0.021739,0.021739,0.065217,...,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739,0.043478,0.021739,0.021739
5,M4P,Central Toronto,Davisville North,0.065217,0.021739,0.021739,0.021739,0.021739,0.021739,0.065217,...,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739,0.043478,0.021739,0.021739
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",0.065217,0.021739,0.021739,0.021739,0.021739,0.021739,0.065217,...,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739,0.043478,0.021739,0.021739
7,M4S,Central Toronto,Davisville,0.065217,0.021739,0.021739,0.021739,0.021739,0.021739,0.065217,...,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739,0.043478,0.021739,0.021739
8,M4T,Central Toronto,"Moore Park, Summerhill East",0.065217,0.021739,0.021739,0.021739,0.021739,0.021739,0.065217,...,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739,0.043478,0.021739,0.021739
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",0.065217,0.021739,0.021739,0.021739,0.021739,0.021739,0.065217,...,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739,0.021739,0.043478,0.021739,0.021739


Top 10 venus for each postcode

In [72]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
2,M4L,East Toronto,"India Bazaar, The Beaches West",Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
3,M4M,East Toronto,Studio District,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
4,M4N,Central Toronto,Lawrence Park,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
5,M4P,Central Toronto,Davisville North,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
7,M4S,Central Toronto,Davisville,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
8,M4T,Central Toronto,"Moore Park, Summerhill East",Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop


#### Clustering

In [91]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/Users/cheungoliver/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [92]:
toronto_merged = df2.copy()
toronto_merged.rename(columns={"Postal Code": "PostalCode"}, inplace = True)


# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop


In [93]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
21,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,0,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
22,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,0,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
23,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,0,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
25,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,0,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
26,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
27,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,0,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
20,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
28,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,0,Coffee Shop,Japanese Restaurant,Café,Art Gallery,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop


#### Visualize the resulting clusters

In [94]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters